In [1]:
# ! pip install transformers datasets
# ! pip install transformers datasets evaluate bleu

In [2]:
import re
import string
import pandas
import evaluate
import unicodedata
import numpy as np
from io import open
from datasets import load_dataset, Dataset
from huggingface_hub import notebook_login
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, pipeline

In [3]:
import torch
torch.cuda.empty_cache()

checkpoint = "t5-large"
folder = "my_awesome_sindarin_model_large"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
source_lang = "en"
# target_lang = "fr"
target_lang = "si"
prefix = "translate English to Sindarin: "
metric = evaluate.load("sacrebleu")
batch_size = 1
# metric = evaluate.load("bleu")

# notebook_login()

c:\Users\cpb5867\.conda\envs\TransConsf\lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-3b automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [4]:
def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=64, truncation=True)
    return model_inputs

In [5]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

In [6]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [7]:
data = pandas.read_csv('data/sindarin-eng.txt' , sep='\t', lineterminator='\n')
data = data.dropna().drop_duplicates()
data = data.reset_index()
data.columns = ['id','si', 'en']

data = data.replace({r'\r': ''}, regex=True)
data = data.replace({r'[^\w\s]': ''}, regex=True)

data['si'] = data['si'].str.lower()
data['en'] = data['en'].str.lower()

data['si'] = data['si'].str.strip()
data['en'] = data['en'].str.strip()

data['si'] = data['si'].str.normalize('NFD').str.encode('ascii', errors='ignore').str.decode('utf-8')
data['en'] = data['en'].str.normalize('NFD').str.encode('ascii', errors='ignore').str.decode('utf-8')

data = data.dropna().drop_duplicates()

data['translation'] = data[['si', 'en']].apply(dict, axis=1)
data.drop(['si', 'en'], axis=1, inplace=True)

print(data)

        id                                        translation
0        0                        {'si': 'ae', 'en': 'hello'}
1        1                          {'si': 'ae', 'en': 'hey'}
2        2                           {'si': 'ae', 'en': 'oh'}
3        3                           {'si': 'ae', 'en': 'ah'}
4        4                  {'si': 'galdol', 'en': 'welcome'}
...    ...                                                ...
3958  3998  {'si': 'ithil eriol dolthol raith erib nan gen...
3959  3999  {'si': 'in elin thinnar', 'en': 'the stars fade'}
3960  4000  {'si': 'i vor danna car ardh gostad', 'en': 't...
3961  4001  {'si': 'dartha dinen a mreithad menel', 'en': ...
3962  4003  {'si': 'padol rath fair padol rath fair', 'en'...

[3963 rows x 2 columns]


In [8]:
# sind = load_dataset('text', data_files={'train': 'data/sindarin-eng.txt'})
# train_ds = Dataset.from_pandas(data)
# sind['train'] = train_ds
# sind = load_dataset({'train': dataset_train})
# sind = sind.remove_columns(["__index_level_0__"])
# print(sind['train'][0])

In [9]:
# books = load_dataset("opus_books", "en-fr")
# books = books["train"].train_test_split(test_size=0.2)
# print(books["train"][0])

books = load_dataset('text', data_files={'train': 'data/sindarin-eng.txt'})
train_ds = Dataset.from_pandas(data)
books['train'] = train_ds
books = books["train"].train_test_split(test_size=0.2)
print(books['train'][0])

tokenized_books = books.map(preprocess_function, batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

torch.cuda.empty_cache()

training_args = Seq2SeqTrainingArguments(
    output_dir= folder,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Found cached dataset text (C:/Users/cpb5867/.cache/huggingface/datasets/text/default-a7038b24a72eb2b5/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)


  0%|          | 0/1 [00:00<?, ?it/s]

{'id': 3655, 'translation': {'en': 'and i smell it in the air', 'si': 'han noston ne gwilith'}, '__index_level_0__': 3627}


Map:   0%|          | 0/3170 [00:00<?, ? examples/s]

Map:   0%|          | 0/793 [00:00<?, ? examples/s]

c:\Users\cpb5867\Documents\CSE-582-FINAL-PROJECT\my_awesome_sindarin_model_3b is already a clone of https://huggingface.co/cpb5867/my_awesome_sindarin_model_3b. Make sure you pull the latest changes with `repo.git_pull()`.


In [10]:
trainer.train()
trainer.push_to_hub()

c:\Users\cpb5867\.conda\envs\TransConsf\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/15850 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB (GPU 0; 24.00 GiB total capacity; 20.60 GiB already allocated; 0 bytes free; 21.29 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
text = "translate English to Sindarin: who is Aragorn"

translator = pipeline("translation", model= folder)
translator(text)

tokenizer = AutoTokenizer.from_pretrained(folder)
inputs = tokenizer(text, return_tensors="pt").input_ids

model = AutoModelForSeq2SeqLM.from_pretrained(folder)
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
tokenizer.decode(outputs[0], skip_special_tokens=True)

'Sindarin, wer Aragorn ist'